In [ ]:
import os
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from datasets import load_dataset

load_dotenv()

In [ ]:
data = load_dataset("microsoft/wiki_qa", split='train')
data = data.to_pandas()
data.head()

In [ ]:
os.environ["OPENAI_APIKEY"] = 'sk-proj-ZIAS0zf7QkcbmX8ukIepT3BlbkFJ2fBzQXWsqU5ZQRqWeEE7'
os.environ["COHERE_API_KEY"] = 'dgRilUFU3exGzYMqCwTYMsl3zcalDTironeQu8jE'
os.environ["WCD_URL"] = "https://science-qa-divghtor.weaviate.network"
os.environ["WCD_API_KEY"] = "dQVdzdjXyULihBdgMebYKeMe92ltMlkwjtJb"
os.environ["QDRANT_API_KEY"] = 'JDUeW-USdj7AAnmT7h8qt61BlPxm5XXZyQDmMBcg1l7QniYOHTjSsQ'
os.environ["QDRANT_URL"] = 'https://87298cc6-470f-4cc0-8e28-aeb973357390.us-east4-0.gcp.cloud.qdrant.io'

In [ ]:
import weaviate
import weaviate.classes as wvc
import os
import json

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.getenv("WCD_URL"),
    auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WCD_API_KEY")),
    headers={
        "X-OpenAI-Api-Key": os.environ["OPENAI_APIKEY"],  # Replace with your inference API key
    }
)

In [ ]:
client.collections.create(
        name="science_qa",
        vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(),  # If set to "none" you must always provide vectors yourself. Could be any other "text2vec-*" also.
         # Ensure the `generative-openai` module is used for generative queries
    )

In [ ]:
sampled_df = data.sample(n=1000, random_state=1)

# Create a list of dictionaries
objs = sampled_df[['question','answer']].to_dict(orient='records')

In [ ]:
objs[:3]

In [ ]:
science_qa = client.collections.get("science_qa")
science_qa.data.insert_many(objs)

In [ ]:
import weaviate
import weaviate.classes as wvc
import os
import json

def export_weaviate(collection_name, cluster_url, api_key, export_file):

    client = weaviate.connect_to_weaviate_cloud(
        cluster_url= cluster_url,
        auth_credentials=weaviate.auth.AuthApiKey(api_key),
    )

    collection = client.collections.get(collection_name)

    items_data = []
    for item in collection.iterator(
        include_vector=True  # If using named vectors, you can specify ones to include e.g. ['title', 'body'], or True to include all
    ):
        item_data = {
            'properties': item.properties,
            'vector': item.vector,
            'uuid': item.uuid.urn.replace('urn:uuid:','')
        }

        items_data.append(item_data)

    with open(export_file, 'w') as json_file:
        json.dump(items_data, json_file, indent=4)

In [ ]:
export_weaviate(collection_name= 'science_qa', cluster_url= os.getenv("WCD_URL"), api_key= os.getenv("WCD_API_KEY"),
                export_file= 'export.json' )

In [ ]:
from qdrant_client import QdrantClient, models


def import_qdrant(url, api_key, collection_name, exported_file):

    with open(exported_file, 'r') as json_file:
        items_data = json.load(json_file)

    properties_list = [item['properties'] for item in items_data]
    vector_default_list = [item['vector']['default'] for item in items_data]
    uuid_list = [item['uuid'] for item in items_data]

    vector_dim = len(vector_default_list[0])

    client = QdrantClient(url=url, api_key= api_key)

    client.recreate_collection(collection_name= collection_name,
                              vectors_config= models.VectorParams(size=vector_dim, distance=models.Distance.COSINE))

    batch_size= 1000

    for i in range(0, len(uuid_list), batch_size):
        batch_uuids = uuid_list[i:i + batch_size]
        batch_properties = properties_list[i:i + batch_size]
        batch_vectors = vector_default_list[i:i + batch_size]

        client.upsert(
            collection_name=collection_name,
            points=models.Batch(
                ids=batch_uuids,
                payloads=batch_properties,
                vectors=batch_vectors,
            ),
        )


In [ ]:
import_qdrant(url= os.getenv("QDRANT_URL")  , api_key= os.getenv("QDRANT_API_KEY") ,
              collection_name= 'science_qa', exported_file= 'items_data.json')